In [ ]:

from datetime import datetime
import numpy as np
import tensorflow as tf
from scipy.ndimage import rotate
from skimage.transform import warp, AffineTransform, resize
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from sklearn.model_selection import train_test_split
from tensorflow.python.ops.numpy_ops.np_dtypes import uint8

#---------------------------- Dataset Load -------------------------------------

print("Loading dataset...")


data = np.load("training_set.npz")

training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

print("Dataset loaded!")
print("Setting classes...")

category_map = {
   0: 0,  # Background
   1: 1,  # Soil
   2: 2,  # Bedrock
   3: 3,  # Sand
   4: 4,  # Big rock
}

#-------------------------- Data Preprocessing ---------------------------------

NUM_CLASSES = len(set(category_map.values()))             # Calculate the correct number of classes after mapping
print("removing outlayers")

mask_alien = y_train[142]                                 # Alien image mask

X_cleaned = []                                            # Initialize lists to store the cleaned dataset
y_cleaned = []

for idx, mask in enumerate(y_train):
   if np.array_equal(mask, mask_alien):                    # Check if the mask is the alien image mask
       continue

   X_cleaned.append(X_train[idx])                          # If it's not a duplicate, add the image and its corresponding mask to the cleaned dataset
   y_cleaned.append(mask)

X_cleaned = np.array(X_cleaned)                            # Convert the cleaned lists to numpy arrays
y_cleaned = np.array(y_cleaned)

print("outlayers removed")
print("Classes setted!")

#------------------------- Train-Test Split ------------------------------------

print("Splitting dataset...")

X_train, X_val, y_train, y_val = train_test_split(X_cleaned, y_cleaned, test_size=0.1, random_state=42, shuffle=True)

print("Data splitted!")

print("Building model...")

#---------------------- U-Net + pyramidal pooling ------------------------------

def pyramidal_pooling_module(input, pool_sizes=[2, 4, 8]):
  pooled1 = tfkl.GlobalAveragePooling2D()(input)
  pooled1 = tfkl.Reshape([ 1, 1, pooled1.shape[-1]])(pooled1)
  upsampled1 = tfkl.UpSampling2D(size=(16,32),interpolation = 'bilinear')(pooled1)      # GAP for global context

  pooled2 = tfkl.AveragePooling2D(pool_size = (8,8))(input)                             # wide pool
  upsampled2 = tfkl.UpSampling2D(size=(8,8),interpolation = 'bilinear')(pooled2)

  pooled3 = tfkl.AveragePooling2D(pool_size = (4,4))(input)                             # mid pool
  upsampled3 = tfkl.UpSampling2D(size=(4,4),interpolation = 'bilinear')(pooled3)

  pooled4 = tfkl.AveragePooling2D(pool_size = (2,2))(input)                             # small pool
  upsampled4 = tfkl.UpSampling2D(size=(2,2),interpolation = 'bilinear')(pooled4)

  concat = tfkl.Concatenate()([upsampled1,upsampled2,upsampled3,upsampled4])

  output = tfkl.Conv2D(64, (1, 1), padding='same')(concat)                              # convolution 1x1 to reduce dimensionality
  conv1 = tfkl.BatchNormalization()(output)
  conv1 = tfkl.ReLU()(conv1)

  return conv1


def unet_block(input_tensor, filters, kernel_size=3, activation='relu', stack=2):

   x = input_tensor

   for i in range(stack):                                                       # Apply a sequence of Conv2D, Batch Normalisation, and Activation layers for the specified number of stacks
       x = tfkl.Conv2D(filters * 2, kernel_size=kernel_size, padding='same')(x)
       x = tfkl.BatchNormalization()(x)
       x = tfkl.Activation(activation)(x)

   # Return the transformed tensor
   return x


def get_unet_model(input_shape=(64, 128, 1), num_classes=NUM_CLASSES):          # UNet model
   input_layer = tfkl.Input(shape=input_shape, name='input_layer')

   down_block_1 = unet_block(input_layer, 16)                                   # Down sampling path
   d1 = tfkl.MaxPooling2D()(down_block_1)

   down_block_2 = unet_block(d1, 32)
   d2 = tfkl.MaxPooling2D()(down_block_2)

   bottleneck = unet_block(d2, 64)                                              # Bottleneck
   ppm = pyramidal_pooling_module(bottleneck)                                   # Pyramid Pooling Module on bottleneck

   u1 = tfkl.UpSampling2D(size=(2,2),interpolation = 'bilinear')(ppm)           # Upsampling path
   u1 = tfkl.Concatenate()([u1, down_block_2])                                  # skip connection
   u1 = unet_block(u1, 32)

   u2 = tfkl.UpSampling2D(size=(2,2),interpolation = 'bilinear')(u1)
   u2 = tfkl.Concatenate()([u2, down_block_1])
   u2 = unet_block(u2, 16)

   output_layer = tfkl.Conv2D(num_classes, kernel_size=1, padding='same', activation="softmax")(u2)    #output softmax layer

   model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
   return model


model = get_unet_model()

#------------------------ IoU metric definition --------------------------------

print("Model built!")

def iou(y_true, y_pred):
   y_true = tf.cast(y_true, tf.float32)
   y_pred = tf.cast(y_pred, tf.float32)

   # Converte i tensori in valori booleani (0 o 1) per il calcolo
   intersection = tf.reduce_sum(y_true * y_pred)
   union = tf.reduce_sum(y_true + y_pred) - intersection

   return intersection / (union + tf.keras.backend.epsilon())


def iou_multi_class(y_true, y_pred, num_classes):
   iou_per_class = []
   y_pred = tf.argmax(y_pred, axis=-1)
   for i in range(num_classes):
       y_true_class = tf.equal(y_true, i)
       y_pred_class = tf.equal(y_pred, i)
       iou_per_class.append(iou(y_true_class, y_pred_class))

   return tf.reduce_mean(iou_per_class)

#------------------------- Model compile and train -----------------------------

print("Compiling model...")

patience = 20
epochs = 1000

print("Parameters setted!")

mean_iou = tfk.metrics.MeanIoU(num_classes=NUM_CLASSES, ignore_class=0, sparse_y_pred=False)  # Define the MeanIoU ignoring the background class
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=[mean_iou])

print("Model compiled!")

early_stopping = tf.keras.callbacks.EarlyStopping(                              # Setup callbacks
   monitor='val_mean_io_u',
   mode='max',
   patience=patience,
   restore_best_weights=True
)

print("Training model...")
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, callbacks=[early_stopping])

#---------------------------- Local Testing ------------------------------------

y_predicted = model.predict(X_val)
final_val_meanIoU = iou_multi_class(y_val, y_predicted, NUM_CLASSES)

print("Model trained!")
print(f'Final validation Mean Intersection Over Union: {final_val_meanIoU}')

import matplotlib.pyplot as plt

#----------------------------- Loss Plot ---------------------------------------

plt.plot(history.history['val_loss'])
plt.title('Validation Loss durante l\'addestramento')
plt.ylabel('Validation Loss')
plt.xlabel('Epoch')
plt.show()

#----------------------------- Model Save --------------------------------------

timestep_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_filename = f"model_{timestep_str}.keras"

model.save(model_filename)
del model

print("Model saved!")

import pandas as pd
from keras.src.saving import load_model

model_filename = "model_241209_222900.keras"
model = load_model(model_filename)


# Function to predict in batches and save directly
def predict_and_save_in_batches(model, X_test, batch_size, filename):
    n_samples = len(X_test)
    n_batches = (n_samples + batch_size - 1) // batch_size  # Compute number of batches

    with open(filename, 'w') as file:
        header_written = False
        for batch_idx in range(n_batches):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, n_samples)

            # Predict the batch
            batch_preds = model.predict(X_test[start_idx:end_idx])
            batch_preds = np.argmax(batch_preds, axis=-1)
            batch_preds = batch_preds.astype('float16')  # Save memory

            # Convert batch predictions to DataFrame
            batch_df = pd.DataFrame(batch_preds.reshape(batch_preds.shape[0], -1))
            batch_df["id"] = np.arange(start_idx, end_idx)
            cols = ["id"] + [col for col in batch_df.columns if col != "id"]
            batch_df = batch_df[cols]

            # Write batch to CSV
            batch_df.to_csv(file, index=False, header=(not header_written))
            header_written = True  # Ensure the header is written only once


# Parameters
batch_size = 256  # Adjust based on your system's memory capacity
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"

# Call the function to predict and save in batches
predict_and_save_in_batches(model, X_test, batch_size, submission_filename)